### Creating Recomender Web App

In [ ]:
# Start by importing useful libraries:
!pip install skl2onnx
import pandas as pd 


In [ ]:
data = pd.read_csv('../data/cleaned_cuisines.csv')
data.head()

In [ ]:
#Remove the first two unnecessary columns and save the remaining data as 'X':

X = data.iloc[:,2:]
X.head()


In [ ]:
#Save the labels as 'y':

y = data[['cuisine']]
y.head()


#### Commence the training routine

We will use the 'SVC' library which has good accuracy.

In [ ]:
#Import the appropriate libraries from Scikit-learn:

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report


In [ ]:
#separate training and test sets:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)


In [ ]:

#Build an SVC Classification model as you did in the previous lesson:
model = SVC(kernel='linear', C=10, probability=True,random_state=0)
model.fit(X_train,y_train.values.ravel())


In [ ]:

# Now, test your model, calling `predict()`:
y_pred = model.predict(X_test)

#Print out a classification report to check the model's quality:
print(classification_report(y_test,y_pred))

### Convert your model to Onnx

Make sure to do the conversion with the proper Tensor number. This dataset has 380 ingredients listed, so you need to notate that number in `FloatTensorType`:


In [ ]:

#convert using a tensor number of 380.
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
    
initial_type = [('float_input', FloatTensorType([None, 380]))]
options = {id(model): {'nocl': True, 'zipmap': False}}


In [ ]:

#Create the onx and store as a file **model.onnx**:
onx = convert_sklearn(model, initial_types=initial_type, options=options)
with open("./model.onnx", "wb") as f:
    f.write(onx.SerializeToString())
    